In [27]:
import numpy as np
import pandas as pd
import json
import requests
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

In [148]:
def read_spotify_data(fp):
    df = pd.read_json(fp)
    return df


def clean_spotify_data(df, cols_to_keep=['ts', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name']):
    df = df[cols_to_keep]

    df = df.rename({'master_metadata_album_artist_name':'Artist',
                    'master_metadata_album_album_name':'Album',
                    'master_metadata_track_name':'Song',
                    'ts':'timestamp'}, axis=1)
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = [x.year for x in df.timestamp]
    df['month'] = [x.month for x in df.timestamp]
    df['day'] = [x.day for x in df.timestamp]
    df['hour'] = [x.hour for x in df.timestamp]

    df['secs_played'] = df['ms_played'] / 1000

    return df

    
def add_cum_sum_secs(df, by_col):   
    df[f'Cumulative_secs_by_{by_col}'] = df.groupby(by_col)['secs_played'].cumsum()

    return df
    
    
def top_table(df, by_col, n=20):
    return df.groupby(by_col).agg({'secs_played': 'sum'}).sort_values('secs_played', ascending=False).reset_index().head(n)


def filter_by_year(df, year):
    return df.query(f'year == {year}')


def trend_graph(df, by_col, top=None):
    df = df.sort_values(by=[by_col, 'timestamp'], ascending=True)
    df = add_cum_sum_secs(df, by_col)
    if top:
        tmp = df.groupby(by_col)['ms_played'].sum().nlargest(top).index
        df = df[df[by_col].isin(tmp)].copy()
    
    fig = px.line(
            df, 
            x='timestamp', 
            y=f'Cumulative_secs_by_{by_col}', 
            color=by_col,
            title=f'Cumulative Time by {by_col}',
            labels={'ts': 'Date', f'Cumulative_ms_by_{by_col}': 'Cumulative Miliseconds', by_col: by_col},
            hover_data={'timestamp': True, 'secs_played': True, f'Cumulative_secs_by_{by_col}': True}
        )
    return fig

In [152]:
dat1723 = read_spotify_data('Streaming_History_Audio_2017-2023_0.json')
dat2325 = read_spotify_data('Streaming_History_Audio_2023-2025_1.json')
spotify = pd.concat([dat1723, dat2325], ignore_index=True)
clean_spotify = clean_spotify_data(spotify)

In [153]:
top_table(clean_spotify, 'Artist')

,Artist,secs_played
0,The Lonely Island,108910.374
1,Fall Out Boy,83216.322
2,Dua Lipa,65189.096
3,First to Eleven,64551.876
4,Mike Posner,63267.996
5,AJR,42965.056
6,The Beach Boys,41921.099
7,Taylor Swift,41572.102
8,Imagine Dragons,38811.473
9,Linkin Park,38149.555


In [154]:
top_table(clean_spotify, 'Song')

,Song,secs_played
0,Snow (Hey Oh),32808.928
1,Attention,27952.463
2,"Danger Zone - From ""Top Gun"" Original Soundtrack",26809.462
3,Face Down,25711.713
4,I Took A Pill In Ibiza - Seeb Remix,25662.303
5,Cooler Than Me - Single Mix,24113.711
6,The Adults Are Talking,22303.983
7,"These Days (feat. Jess Glynne, Macklemore & Da...",19937.447
8,3 Foot Tall,19427.982
9,Have It All,19376.406


In [155]:
trend_graph(clean_spotify, 'Artist', 10)

## 2025

In [156]:
year_25 = filter_by_year(clean_spotify, 2025)
top_table(year_25, 'Artist')

,Artist,secs_played
0,The Lonely Island,96887.324
1,Coldplay,19155.264
2,Taylor Swift,17163.947
3,The Unauthorized Bash Brothers Experience,16741.269
4,OCT,14570.043
5,Nickelback,14248.600
6,Noah Kahan,12128.578
7,Sabrina Carpenter,11823.189
8,Dua Lipa,11326.674
9,vaultboy,11021.344


In [133]:
top_table(year_25, 'Song')

,Song,secs_played
0,Snow (Hey Oh),8062.428
1,Rocky,7334.319
2,Stolen Dance,6927.086
3,I'm On A Boat,6906.714
4,Sweet Child O' Mine,6484.043
5,Mona Lisa,6418.519
6,Don't Touch My Clogs,6278.960
7,Paralyzer,5915.968
8,Invincible,5741.965
9,Burn It to the Ground,5570.837


In [149]:
trend_graph(year_25, 'Artist', 15)

In [141]:
trend_graph(year_25, 'Song', 10)

## 2024

In [157]:
year_24 = filter_by_year(clean_spotify, 2024)

In [139]:
top_table(year_24, 'Artist')

,Artist,secs_played
0,Noah Kahan,16251.220
1,Taylor Swift,13414.517
2,Dua Lipa,12894.045
3,Linkin Park,12447.122
4,The Lonely Island,12023.050
5,First to Eleven,11154.343
6,Mike Posner,9389.667
7,AJR,9272.501
8,Nickelback,9173.191
9,Sabrina Carpenter,7731.592


In [140]:
top_table(year_24, 'Song')

,Song,secs_played
0,Read your Mind,6417.291
1,Stolen Dance,6304.031
2,Face Down,5995.792
3,Someone To You,5278.924
4,Burn It to the Ground,4622.676
5,False Confidence,4339.630
6,Cooler Than Me - Single Mix,4262.886
7,Attention,4191.146
8,Sweet Child O' Mine,4105.484
9,The Feeling - Lost Frequencies & Andromedik De...,4102.517


In [145]:
trend_graph(year_24, 'Artist', 10)

In [144]:
trend_graph(year_24, 'Song', 10)

## Which songs were skipped the most

In [175]:
cols = ['ts', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'skipped', 'reason_end']
skipped = clean_spotify_data(spotify, cols)
skipped.groupby('Song').agg({'skipped':'sum'}).sort_values('skipped', ascending=False).head(20).reset_index()


,Song,skipped
0,aftermath,37
1,Sunshine,34
2,Close To You,33
3,we had everything...,32
4,rocket science,32
5,everything sucks,29
6,Break My Heart,29
7,I Can Do It With a Broken Heart,29
8,Home,29
9,Weak,29


In [176]:
skipped['reason_end'].value_counts()

reason_end
trackdone                       16035
fwdbtn                           6156
endplay                          3304
logout                            996
unexpected-exit-while-paused      837
backbtn                           246
trackerror                         42
unexpected-exit                    12
unknown                             7
remote                              5
Name: count, dtype: int64

In [181]:
# skipped.query("reason_end == 'endplay'").groupby('Song').agg({'reason_end':'count'}).sort_values('reason_end', ascending=False).head(10).reset_index()

## Which songs were picked the most

In [182]:
cols = ['ts', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'skipped', 'reason_start']
picked = clean_spotify_data(spotify, cols)
picked['reason_start'].value_counts()


reason_start
trackdone     16299
fwdbtn         6171
clickrow       2952
appload         998
playbtn         807
backbtn         249
unknown          98
trackerror       61
remote            5
Name: count, dtype: int64

In [184]:
picked.query("reason_start == 'playbtn'").groupby('Song').agg({'reason_start':'count'}).sort_values('reason_start', ascending=False).head(20).reset_index()

,Song,reason_start
0,"Danger Zone - From ""Top Gun"" Original Soundtrack",17
1,Thnks fr th Mmrs,11
2,rocket science,10
3,everything sucks,10
4,Cool,8
5,Face Down,8
6,Ain't No Rest for the Wicked,7
7,Read your Mind,6
8,Tongue Tied,6
9,Talk Too Much,6
